# Creating sub-agent 1: Github Assistant
In this folder we will create the second sub-agent the github assistant

The github assistant handles GitHub operations like finding and searching pull requests while letting other agents handle their specific domains.

The agent architecture looks as following:
![Architecture](images/architecture.png)

The agent has an action group
## Prerequisites
Before starting, let's install the required boto3 libraries. Since this is a preview, the required libraries are being installed from the provided `whl` files

**Important:** this command will result in some conflict errors. It is ok to ignore those for now! We will check the pip versions in the next step

In [ ]:
!python3 -m pip install --force-reinstall --no-cache -q -r requirements.txt

## Import required libraries
Next we will import the required libraries. We will also import some support functions available in the parent directory. Those functions are:
- `create_lambda_layer`: to create a lambda layer with the requirements for the agent
- `create_agent`: helps you to create the necessary IAM permissions and Bedrock agetns based on the agent's name, instructions, foundation models, descriptions and other properties
- `invoke_agent_helper`: helps you to invoke your agent using invoke_agent

You can see the implementation of both functions in the parent directory

In [ ]:
import time
import sys
import os
import configparser

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from agents import create_lambda_layer, create_agent, invoke_agent_helper

#load config
config = configparser.RawConfigParser()
config.read('../devops.properties')


## Create a Lambda layer
Create a lambda layer with the following packages
```
requests
pandas
numpy
```

In [ ]:
agent_name = 'github-assistant'
requiments = 'lambda_requirements.txt'
lambda_layer_arn = create_lambda_layer(agent_name, requiments)
lambda_layer_arn

## Defining agent configuration

Let's now define the configuration for our grafana assistant. Let's use the follow instructions to create our agent:

```
You are an agent specialized in GitHub operations, focusing on finding and analyzing pull requests and related repository information. 

Primary Tasks: Search and find pull requests based on various criteria

When to Use Me:
- When you need to find specific pull requests

What I Don't Do:
- Code analysis 
- Security assessments 
- Documentation tasks 
- Deployment operations
- Anything else

Remember: I focus solely on GitHub operations
```

For this agent, we will use `Claude 3 Haiku` model in order to provide faster answers to the user.

In [ ]:
agent_foundation_model = "anthropic.claude-3-haiku-20240307-v1:0"
agent_instruction = """You are an agent specialized in GitHub operations, focusing on finding and analyzing pull requests and related repository information. 

Primary Tasks: Search and find pull requests based on various criteria

When to Use Me:
- When you need to find specific pull requests

What I Don't Do:
- Code analysis 
- Security assessments 
- Documentation tasks 
- Deployment operations
- Anything else

Remember: I focus solely on GitHub operations"""
agent_description = "Agent to manage github actions"

agent_functions = [
    {
        'name': 'get_git_hub_pull_requests',
        'description': 'Gets number of pull requests done in a repo owner by a particular owner',
        'parameters': {
            "owner": {
                "description": "owner of the github repo",
                "required": True,
                "type": "string"
            },
            "repo": {
                "description":"repo name in github you want to find pull requests for",
                "required": True,
                "type": "string"
            }
        }
    }
]

action_group_config = {
    'name': 'GithubActionGroup',
    'description': 'Action group to manage github actions',
    'lambda_function_name': f'{agent_name}-lambda',
    'lambda_file_path': 'lambda_function.py',
    'lambda_layers': [lambda_layer_arn],
    'functions': agent_functions,
    'environment': {
        'Variables': {
            'github_token': config.get('github', 'github.token')
        }
    }
}

## Create agent
Next we will create the agent with the provided information

In [ ]:
github_agent_id, github_assistant_agent_alias_id, github_assistant_agent_alias_arn = create_agent(
    agent_name,
    agent_instruction,
    agent_foundation_model=agent_foundation_model,
    agent_description=agent_description,
    action_group_config=action_group_config
)
time.sleep(20)

## Getting details from the agent

Let's take a look at the details from the created agent. We will need its `github_assistant_agent_alias_arn` to associate the agent with its supervisor agent

In [ ]:
github_agent_id, github_assistant_agent_alias_id, github_assistant_agent_alias_arn

## Testing Agent
Now that we've created the agent, let's test it by using our `invoke_agent_helper` function

### Getting started
First let's get the history of alerts for app1

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """get me all github related pull requests for owner mttanke for repo amazon-bedrock-samples"""
response = invoke_agent_helper(
    query, session_id, github_agent_id, github_assistant_agent_alias_id, enable_trace=True
)
print(response)

## Saving the information
Let's now save some information so that we can use this agent as a sub-agent of our traveler assistant

In [ ]:
%store github_agent_id
%store github_assistant_agent_alias_id
%store github_assistant_agent_alias_arn

## Next steps
Next let's create our other sub-agents for github